In [27]:
%%javascript

function getDrops (total, max, min) {
    const ticksCount = max - min
    const dropsCount = ticksCount - total
    const drops = []
    while (drops.length < dropsCount) {
        const rnd = Math.floor(Math.random() * ticksCount) + min
        if (drops.indexOf(rnd) !== -1) continue
        drops.push(rnd)
    }
    return drops
}

function getLogs ({
    sentences,
    dateStart,
    dateEnd,
    padChar,
    pidMin,
    pidMax,
    lineLength,
    lineCount,
    statuses
}) {
    const getPid = () => Math.floor(Math.random() * (pidMax - pidMin)) + pidMin
    const getStatus = () => statuses[Math.floor(Math.random() * statuses.length)]
    const getData = () => sentences[Math.floor(Math.random() * sentences.length)]
    const drops = getDrops(lineCount, dateEnd, dateStart)

    const logs = []
    for (let tick = dateStart; tick < dateEnd; tick++) {
        if (drops.indexOf(tick) !== -1) continue
        const d = new Date(tick * 1000)
        const date = [
            d.getUTCFullYear(),
            (d.getUTCMonth() + 1).toString().padStart(2, '0'),
            (d.getUTCDate()).toString().padStart(2, '0')
        ].join('')
        const time = [
            d.getUTCHours().toString().padStart(2, '0'),
            d.getUTCMinutes().toString().padStart(2, '0'),
            d.getUTCSeconds().toString().padStart(2, '0')
        ].join(':')

        let msg = [
            date,
            time,
            getPid(),
            getStatus(),
            getData()
        ].join('|')

        msg = msg.substr(0, lineLength - 1)
        msg += '|'
        msg = msg.padEnd(lineLength, padChar)

        logs.push(msg)
    }
    return logs.join('\r\n')
}

async function getSentences (url) {
    const json = await fetch(url).then(res => res.json())
    const text = json.parse.text['*']
    const frag = document.implementation.createHTMLDocument('preview')
    const div = frag.createElement('div')
    div.innerHTML = text

    let lines = [];
    ['//p', '//ul/li'].forEach(expr => {
        const res = document.evaluate(expr, div)
        let found
        while (found = res.iterateNext()) {
            const foundLines = found.textContent
                .replace(/\[[^\]]+\]/g, '')
                .trim()
                .split('. ')
                .filter(line =>
                    line.match(/^[A-Z]/) &&
                    line.split(/\s+/g).length > 2
                ).map(line =>
                    line
                        .replace(/\W+/g, ' ')
                        .replace(/[.:\n]$/, '') + '.'
                )
            lines.push(...foundLines)
        }
    })
    return lines
}

;(async function () {
    const url = 'https://en.wikipedia.org/w/api.php?action=parse&prop=text&page=Amazon_S3&format=json'
    const sentences = await getSentences(`https://cors.io/?${url}`)
    const date = 1325376000

    window.data = getLogs({
        sentences,
        dateStart: date + 3600 * 9,
        dateEnd: date + 3600 * (9 + 3),
        padChar: "X",
        statuses: ['OK', 'TEMP', 'PERM'],
        pidMin: 3000,
        pidMax: 5000,
        lineLength: 500,
        lineCount: 10000
    })
})()

<IPython.core.display.Javascript object>

In [29]:
%%javascript

require.config({
    paths: {
        d3: 'https://d3js.org/d3.v5.min',
        bc: 'https://cdn.jsdelivr.net/npm/britecharts@2/dist/bundled/britecharts.min'
    }
})

element.append(`
<link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/npm/britecharts@2/dist/css/britecharts.min.css">
<style>
#graph {
    background-color: red;
}
</style>
`)

require(['d3', 'bc'], function(d3, bc) {
    function parseRows (data) {
        const parseDateTime = d3.timeParse('%Y%m%d %H:%M:%S')
        return d3.dsvFormat('|')
            .parseRows(data, (row) => ({
                date: parseDateTime(`${row[0]} ${row[1]}`),
                status: row[3],
                message: row[4]
            }))
    }
    
    function dropsByMinuteRow (row) {
        return Object.keys(row).reduce((drops, key) => {
            return key === 'date'
                ? drops
                : drops - row[key]
        }, 60)
    }

    function groupByMinute (parsedData) {
        const formatDateTime = d3.timeFormat('%s')
        const groupedData = []
        let groupedDataRow = {}
        for (const {date, status} of parsedData) {
            const ts = Math.floor(formatDateTime(date) / 60) * 60

            if (groupedDataRow.date && groupedDataRow.date !== ts) {
                groupedDataRow.drops = dropsByMinuteRow(groupedDataRow)
                groupedData.push(groupedDataRow)
                groupedDataRow = {}
            }

            groupedDataRow.date = ts
            typeof groupedDataRow[status] === 'undefined'
                ? groupedDataRow[status] = 1 
                : groupedDataRow[status]++
        }

        groupedDataRow.drops = dropsByMinuteRow(groupedDataRow)
        groupedData.push(groupedDataRow)
        return groupedData
    }
        
    const groupedData = groupByMinute(
        parseRows(data)
    )
       
    const dataset = groupedData.reduce((a, v) => {
        for (const key of Object.keys(v)) {
            if (key === 'date') continue
            a.push({
                date: new Date(v.date * 1000),
                name: key,
                count: v[key]
            })
        }
        return a
    }, [])
    
    const stackedArea = bc.stackedArea()
    const container = d3.select(element.get(0))
    const containerWidth = container.node().getBoundingClientRect().width
    const chartTooltip = bc.tooltip()
    
    stackedArea
        .isAnimated(true)
        .areaOpacity(0.9)
        .tooltipThreshold(600)
        .width(containerWidth)
        .xAxisFormat('custom')
        .xAxisCustomFormat('%H:%M')
        .xTicks(12)
        .yTicks(4)
        .valueLabel('count')
        .keyLabel('name')
        .topicsOrder(['OK', 'TEMP', 'PERM', 'drops'])
        .on('customMouseOver', chartTooltip.show)
        .on('customMouseMove', chartTooltip.update)
        .on('customMouseOut', chartTooltip.hide)
    
    stackedArea.colorSchema(['24BC14', 'ECD748', 'EA644A', '000000'])
    container
        .datum(dataset)
        .call(stackedArea)
    
    chartTooltip
        .topicLabel('values')
        .title('Sample time')
        .dateFormat(chartTooltip.axisTimeCombinations.CUSTOM)
        .dateCustomFormat('%H:%M')
        .topicsOrder(['OK', 'TEMP', 'PERM', 'drops'])
    
    d3.select('.metadata-group .vertical-marker-container')
        .attr('opacity', '0.8')
        .datum([])
        .call(chartTooltip)
})


<IPython.core.display.Javascript object>